In [8]:

%load_ext autoreload
%autoreload 2

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import numpy as np
from src.motion_refiner_4D import Motion_refiner, MAX_NUM_OBJS
from src.config import *
# base_folder = "/home/tum/data/"
# base_folder = "/home/arthur/local_data/" 

traj_n = 40
mr = Motion_refiner(load_models=True ,traj_n = traj_n, locality_factor=True, clip_only=False)
feature_indices, obj_sim_indices, obj_poses_indices, traj_indices = mr.get_indices()
embedding_indices = np.concatenate([feature_indices,obj_sim_indices, obj_poses_indices])

loading BERT model... 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


done
loading CLIP model... done
DEVICE:  cuda


In [10]:
dataset_name = "4D_100k_scaping_factor"

X,Y, data = mr.load_dataset(dataset_name, filter_data = True, base_path=base_folder+"data/")
X_train, X_test, X_valid, y_train, y_test, y_valid, indices_train, indices_test, indices_val = mr.split_dataset(X, Y, test_size=0.2, val_size=0.1)


loading dataset:  4D_100k_scaping_factor ...done
raw X: (100000, 953) 	Y: (100000, 160)
filtered X: (99092, 953) 	Y: (99092, 160)
Train X: (69363, 953) 	Y: (69363, 160)
Test  X: (19819, 953) 	Y: (19819, 160)
Val   X: (9910, 953) 	Y: (9910, 160)


In [12]:
from src.functions import *

n = -4
trajs_x = mr.prepare_x(X_test)[:n,MAX_NUM_OBJS:,:]
trajs_y = list_to_wp_seq(y_test,d=4)[:n,:,:]

In [13]:
metrics, metrics_h = compute_metrics(trajs_x, trajs_y)

pcm:	 nan
dfd:	 0.14606799108660493
area:	 0.027892437444718723
cl:	 nan
dtw:	 3.789714883393604
mae:	 0.02784615995780797
mse:	 0.005119454187622462


In [43]:
from tqdm import tqdm
change_img_base=['/mnt/tumdata/image_dataset',image_dataset_folder]

all_image_paths = []
for i, d in tqdm(enumerate(data)):
    image_paths = d["image_paths"]

    if not change_img_base is None:
        for ti in range(len(image_paths)):
            all_image_paths.append(image_paths[ti].replace(change_img_base[0], change_img_base[1]))
    
print(len(all_image_paths))
set_res = set(all_image_paths) 
print("The unique elements of the input list using set():\n") 
all_image_paths = (list(set_res))
print(len(all_image_paths))

# for item in list_res: 
#     print(item) 
# img_paths = [dd["image_paths"] for d in data]

99092it [00:00, 687159.06it/s]

395345
The unique elements of the input list using set():

9290


In [66]:
import json
obj_library = {}
with open(image_dataset_folder+"imagenet1000_clsidx_to_labels.txt") as f:
    obj_library = json.load(f)

obj_names = []
for k,v in obj_library.items():
    obj_names= obj_names+obj_library[k] 
print(len(obj_names))

1858


In [70]:
import os
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm
from PIL import Image


# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)


# def load_image(img_paths):
#     img = 
#     return img

def image_loader(lst, n):
    for i in range(0, len(lst), n):
        b = lst[i:i + n]
        classes = [c.split("/")[-2] for c in b]
        images = torch.concat([preprocess(Image.open(im)).unsqueeze(0) for im in b])
        yield classes,images
def get_image_features(image_paths):
    all_features = []

    with torch.no_grad():
        for classes, images in image_loader(image_paths,100):
            print(images.shape)
            features = model.encode_image(images.to(device))
            all_features.append(features)
    return torch.cat(all_features).cpu().numpy()

def text_loader(lst, n):
    for i in range(0, len(lst), n):
        yield clip.tokenize(lst[i:i + n]).to(device)
def get_text_features(text):
    all_features = []

    with torch.no_grad():
        for t in text_loader(text,100):
            features = model.encode_text(t.to(device))
            all_features.append(features)
    return torch.cat(all_features).cpu().numpy()

all_obj_names_features = get_text_features(obj_names)

# all_features = get_image_features(all_image_paths)
# all_features.shape


In [72]:
import json
obj_names_clip_emb = {}
for f, p in zip(all_obj_names_features,obj_names):
    obj_names_clip_emb[p] = f.tolist()

with open(image_dataset_folder+'obj_names_clip_emb.json', 'w', encoding='utf-8') as f:
    json.dump(obj_names_clip_emb, f, ensure_ascii=False, indent=4)

In [50]:
import json
images_clip_emb = {}
for f, p in zip(all_features,all_image_paths):
    images_clip_emb[p] = f.tolist()
    
with open(image_dataset_folder+'images_clip_emb.json', 'w', encoding='utf-8') as f:
    json.dump(images_clip_emb, f, ensure_ascii=False, indent=4)

In [80]:
from src.motion_refiner_4D import Motion_refiner

mr = Motion_refiner(traj_n = 40, clip_only=True, locality_factor=True, poses_on_features=True, load_precomp_emb=True)


TypeError: __init__() got an unexpected keyword argument 'load_precomp_emb'

In [ ]:
X_,Y_ = mr.prepare_data(data,deltas=False, change_img_base=['/mnt/tumdata/image_dataset',args.image_dataset_dir])


In [53]:
images_clip_emb2={}
with open(image_dataset_folder+"images_clip_emb.json") as f:
    images_clip_emb2 = json.load(f)
print(len(images_clip_emb2.keys()))

9290


In [8]:
X,Y = mr.prepare_data(data[:5],deltas=False, change_img_base=['/mnt/tumdata/image_dataset',image_dataset_folder])

text_features  (5, 512)


1it [00:00,  4.25it/s]

text_clip_features  torch.Size([1, 512])


2it [00:38, 22.51s/it]

text_clip_features  torch.Size([1, 512])


2it [01:03, 31.55s/it]


KeyboardInterrupt: 

## Dynamic time warping (DTW)

# test custom loss

In [2]:
import tensorflow as tf

In [53]:
dtype = tf.float32
y_true =  tf.constant([[[1.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0]],
                        [[1.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0]]],dtype=dtype)
y_pred =  tf.constant([[[0.5, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0]],
                        [[2.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0],[1.0, 1.0, 1.0, 1.0]]],dtype=dtype)

y_true_abs = tf.norm(y_true, ord='euclidean', axis=-1)
y_pred_abs = tf.norm(y_pred, ord='euclidean', axis=-1)

y_true = tf.expand_dims(y_true, -1)
y_pred = tf.expand_dims(y_pred, -1)

y_pred  = tf.transpose(y_pred, [0, 1, 3,2])

tf.print(tf.shape(y_pred))
tf.print(tf.shape(y_true))

a = tf.squeeze(tf.matmul(y_pred, y_true),[2, 3])
b = tf.multiply(y_true_abs,y_pred_abs)
angle_diff = tf.add(1.0,-tf.math.divide(a,b))
lenght_diff = tf.math.divide(tf.math.sqrt(tf.square(y_true_abs-y_pred_abs)),tf.add(y_true_abs,y_pred_abs))

k1 = 0.5
k2 = 0.5

loss_LDA = tf.add(tf.math.multiply(k1,lenght_diff), tf.math.multiply(k2*1/2,angle_diff))

[2 3 1 4]
[2 3 4 1]
matmul
-----------------
[2 3]
[[0.0332502946 0 0]
 [0.0832713246 0 0]]
